# Step 1: Simulation

## a) Run the Monte Carlo simulation
The simulations are called from a standalone python file, as they take quite long.
It is recommended to run them in a `screen` with

```sh
nice python 01-simulate.py
```

Note: Supervise running simulations with 
```sh
while true; do clear; tail -n1 output/*.simu.log | grep '>>>'; sleep 10; done
```

## b) Check for successful execution

In [1]:
distances = [15, 35]
doubleplanes = [8, 12, 30]
energies = [200, 600, 1000]
erels = [100, 500]
neutrons = [1, 2, 3, 4, 5, 6]
physicss = ["bert", "bic", "inclxx"]
scenarios = ["air", "vacuum"]

In [6]:
import pandas as pd
import numpy as np
import joblib
import os
import ROOT

ROOT.ROOT.EnableThreadSafety()

In [7]:
def processed_events(distance, doubleplane, energy, erel, neutron, physics, scenario):
    filepattern = "output/%s-%s/%dm_%ddp_%dAMeV_%dkeV_%dn.%s.root"
    filename = filepattern % (
        physics.lower(),
        scenario,
        distance,
        doubleplane,
        energy,
        erel,
        neutron,
        "simu",
    )
    if os.path.isfile(filename):
        try:
            tfile = ROOT.TFile.Open(filename)
            ttree = tfile.Get("evt")
            num_events = int(ttree.GetEntries())
            return (filename, num_events)
        except:
            pass
    return (filename, 0)

In [8]:
evts = pd.DataFrame(
    joblib.Parallel(n_jobs=-1, backend="multiprocessing", verbose=1)(
        joblib.delayed(processed_events)(
            distance=distance,
            doubleplane=doubleplane,
            energy=energy,
            erel=erel,
            neutron=neutron,
            physics=physics,
            scenario=scenario,
        )
        for distance in distances
        for energy in energies
        for doubleplane in doubleplanes
        for neutron in neutrons
        for erel in erels
        for physics in physicss
        for scenario in scenarios
    )
)

[Parallel(n_jobs=-1)]: Using backend MultiprocessingBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done 136 tasks      | elapsed:    5.4s
[Parallel(n_jobs=-1)]: Done 386 tasks      | elapsed:    8.1s
[Parallel(n_jobs=-1)]: Done 736 tasks      | elapsed:   12.0s
[Parallel(n_jobs=-1)]: Done 1186 tasks      | elapsed:   17.4s
[Parallel(n_jobs=-1)]: Done 1296 out of 1296 | elapsed:   19.1s finished


In [9]:
pd.options.display.max_rows = 5
evts.columns = ["File", "Events"]
evts.sort_values(by=["Events", "File"], ascending=[True, True], inplace=True)
evts.style.hide_index().format({"Events": "{:d}"})
display(evts)

,File,Events
504,output/bert-air/15m_12dp_1000AMeV_100keV_1n.si...,10000
516,output/bert-air/15m_12dp_1000AMeV_100keV_2n.si...,10000
...,...,...
923,output/inclxx-vacuum/35m_8dp_600AMeV_500keV_5n...,10000
935,output/inclxx-vacuum/35m_8dp_600AMeV_500keV_6n...,10000
